# Access Computation

This tutorial demonstrates how to compute access.

## Setup

In [ ]:
import numpy as np
import pandas as pd

import plotly.graph_objs as go

from ostk.mathematics.objects import RealInterval

from ostk.physics.units import Length
from ostk.physics.units import Angle
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.time import Time
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.coordinate.spherical import AER
from ostk.physics.coordinate import Position
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth

from ostk.astrodynamics import Trajectory
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory.orbit.models import Kepler
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE
from ostk.astrodynamics.trajectory.orbit.models import SGP4
from ostk.astrodynamics.trajectory.orbit.models.sgp4 import TLE
from ostk.astrodynamics import Access
from ostk.astrodynamics.access import Generator as AccessGenerator

---

## Access

An access represents an object-to-object visibility period.

In this example, let's compute accesses between a fixed position on the ground and a satellite in LEO.

## Environment

Let's setup an environment (which describes where planets are, etc...):

In [ ]:
environment = Environment.default() ;

### Origin

Let's define a fixed ground position, using its geographic coordinates:

In [ ]:
latitude = Angle.degrees(50.0)
longitude = Angle.degrees(20.0)
altitude = Length.meters(30.0)

In [ ]:
from_lla = LLA(latitude, longitude, altitude)

In [ ]:
from_position = Position.meters(from_lla.to_cartesian(Earth.equatorial_radius, Earth.flattening), Frame.ITRF())

And derive a trajectory, fixed at that position:

In [ ]:
from_trajectory = Trajectory.position(from_position)

### Target

Let's consider a satellite in **Low-Earth Orbit**.

In [ ]:
earth = environment.access_celestial_object_with_name("Earth")

We can define its orbit with **Classical Orbital Elements**:

In [ ]:
a = Earth.equatorial_radius + Length.kilometers(500.0)
e = 0.000
i = Angle.degrees(97.8893)
raan = Angle.degrees(100.372)
aop = Angle.degrees(0.0)
nu = Angle.degrees(0.0201851)

coe = COE(a, e, i, raan, aop, nu)

... and by using a **Keplerian** orbital model:

In [ ]:
epoch = Instant.date_time(DateTime(2018, 1, 1, 0, 0, 0), Scale.UTC)

keplerian_model = Kepler(coe, epoch, earth, Kepler.PerturbationType.J2)

Or with a **Two-Line Element** (TLE) set:

In [ ]:
tle = TLE(
    "ISS (ZARYA)",
    "1 25544U 98067A   18268.86272795  .00002184  00000-0  40781-4 0  9990",
    "2 25544  51.6405 237.0010 0003980 205.4375 242.3358 15.53733046134172"
)

... along with its associated **SGP4** orbital model:

In [ ]:
sgp4_model = SGP4(tle)

Below, we select which orbital model to use:

In [ ]:
orbital_model = keplerian_model
# orbital_model = sgp4_model

We then obtain the satellite orbit (which is a **Trajectory** object):

In [ ]:
satellite_orbit = Orbit(orbital_model, earth)

Alternatively, the **Orbit** class can provide some useful shortcuts (for usual orbit types):

In [ ]:
epoch = Instant.date_time(DateTime(2018, 1, 1, 0, 0, 0), Scale.UTC)

satellite_orbit = Orbit.sun_synchronous(epoch, Length.kilometers(500.0), Time(12, 0, 0), earth)

### Access

Now that the origin and the target trajectories are well defined, we can compute the **Access**.

Let's first define an **analysis interval**:

In [ ]:
start_instant = Instant.date_time(DateTime.parse("2018-01-01 00:00:00"), Scale.UTC) ;
end_instant = Instant.date_time(DateTime.parse("2018-01-10 00:00:00"), Scale.UTC) ;

interval = Interval.closed(start_instant, end_instant) ;

Then, using an **Access Generator**, we can compute the accesses within the intervals of interest:

In [ ]:
azimuth_range = RealInterval.closed(0.0, 360.0) # [deg]
elevation_range = RealInterval.closed(20.0, 90.0) # [deg]
range_range = RealInterval.closed(0.0, 10000e3) # [m]

# Access generator with Azimuth-Range-Elevation constraints

access_generator = AccessGenerator.aer_ranges(azimuth_range, elevation_range, range_range, environment)

In [ ]:
accesses = access_generator.compute_accesses(interval, from_trajectory, satellite_orbit)

And format the output using a dataframe:

In [ ]:
accesses_df = pd.DataFrame([[str(access.get_type()), repr(access.get_acquisition_of_signal()), repr(access.get_time_of_closest_approach()), repr(access.get_loss_of_signal()), float(access.get_duration().in_seconds())] for access in accesses], columns=['Type', 'AOS', 'TCA', 'LOS', 'Duration'])

### Output

Print accesses:

In [ ]:
accesses_df

Let's calculate the geographic coordinate of the satellite, during access:

In [ ]:
def compute_lla (state):
    
    lla = LLA.cartesian(state.get_position().in_frame(Frame.ITRF(), state.get_instant()).get_coordinates(), Earth.equatorial_radius, Earth.flattening)

    return [float(lla.get_latitude().in_degrees()), float(lla.get_longitude().in_degrees()), float(lla.get_altitude().in_meters())]

def compute_aer (instant, from_lla, to_position):
    
    nedFrame = earth.get_frame_at(from_lla, Earth.FrameType.NED)

    fromPosition_NED = from_position.in_frame(nedFrame, instant)
    sunPosition_NED = to_position.in_frame(nedFrame, instant)

    aer = AER.from_position_to_position(fromPosition_NED, sunPosition_NED, True)
            
    return [float(aer.get_azimuth().in_degrees()), float(aer.get_elevation().in_degrees()), float(aer.get_range().in_meters())]

def compute_time_lla_aer_state (state):
    
    instant = state.get_instant()
    
    lla = compute_lla(state)
    aer = compute_aer(instant, from_lla, state.get_position().in_frame(Frame.ITRF(), state.get_instant()))

    return [instant, lla[0], lla[1], lla[2], aer[0], aer[1], aer[2]]

def compute_trajectory_geometry (aTrajectory, anInterval):

    return [compute_lla(state) for state in aTrajectory.get_states_at(anInterval.generate_grid(Duration.minutes(1.0)))]

def compute_access_geometry (access):

    return [compute_time_lla_aer_state(state) for state in satellite_orbit.get_states_at(access.get_interval().generate_grid(Duration.seconds(1.0)))]

In [ ]:
satellite_orbit_geometry_df = pd.DataFrame(compute_trajectory_geometry(satellite_orbit, interval), columns=['Latitude', 'Longitude', 'Altitude'])

In [ ]:
satellite_orbit_geometry_df.head()

In [ ]:
access_geometry_dfs = [pd.DataFrame(compute_access_geometry(access), columns=['Time', 'Latitude', 'Longitude', 'Altitude', 'Azimuth', 'Elevation', 'Range']) for access in accesses] ;

In [ ]:
def get_max_elevation (df):
    
    return df.loc[df['Elevation'].idxmax()]['Elevation']

And plot the geometries onto a map:

In [ ]:
data = []

# Target geometry

data.append(
    dict(
        type = 'scattergeo',
        lon = [float(longitude.in_degrees())],
        lat = [float(latitude.in_degrees())],
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'orange'
        )
    )
)

# Orbit geometry

data.append(
    dict(
        type = 'scattergeo',
        lon = satellite_orbit_geometry_df['Longitude'],
        lat = satellite_orbit_geometry_df['Latitude'],
        mode = 'lines',
        line = dict(
            width = 1,
            color = 'rgba(0, 0, 0, 0.1)',
        )
    )
)

# Access geometry

for access_geometry_df in access_geometry_dfs:
    
    data.append(
        dict(
            type = 'scattergeo',
            lon = access_geometry_df['Longitude'],
            lat = access_geometry_df['Latitude'],
            mode = 'lines',
            line = dict(
                width = 1,
                color = 'red',
            )
        )
    )
    
layout = dict(
        title = None,
        showlegend = False,
        height = 1000,
        geo = dict(
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )
    
figure = go.Figure(data = data, layout = layout)

figure.show()

---